# 正则表达式与文本处理

* 字符串函数
* 正则表达式
* 遍历字符序列，进行处理

## 回顾：字符串函数

* 类型判断
* 大小写转换
* 字符串拆分与拼接
* 搜索与替换
* 翻译与转换

假设我们有一段英文文本，希望把这段文本中的单词全部提取出来，放在一个列表中，如何操作？

字符串的split方法可以帮助我们完成这个任务。但不完美。

In [1]:
sent = "a secret system, a machine, that spies on you every hour of every day."
print(sent.split())

['a', 'secret', 'system,', 'a', 'machine,', 'that', 'spies', 'on', 'you', 'every', 'hour', 'of', 'every', 'day.']


In [2]:
import re
print(re.findall('\w+', sent))

['a', 'secret', 'system', 'a', 'machine', 'that', 'spies', 'on', 'you', 'every', 'hour', 'of', 'every', 'day']


## 正则表达式

* 正则表达式构成
* 字符类和预定义字符类
* 重复次数限定符
* 贪婪匹配与懒惰匹配
* 分组和引用
* 匹配选项

In [3]:
import re

Python标准库的re模块实现了正则表达式处理的功能。使用之前需要先导入。
```
  import re
```
在学习这个模块之前，我们先看一个例子。假设我们想从文本中提取IP地址，可以这样做。

In [4]:
# 例子： 提取文本中的IP地址
click = "you are visiting this website from ip: 202.38.102.75"
re.findall("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", click)

['202.38.102.75']

我们以下主要针对英文文本进行讲解。当然正则表达式也可以处理中文文本。

若不限定采用ASCII码字母匹配(re.A选项)，`'\w'`可以匹配汉字。或者我们使用`[\u4e00-\u9fa5]`显式匹配一个汉字。

例如提取文本中的路名和门牌号信息，地址形式：区xx路xx号

In [5]:
addr = "合肥市包河区马鞍山南路588号金地国际城，安徽省合肥市经济技术开发区莲花路1002号"
print(re.findall('区((\w+?路)(\w+?号))', addr))
#print(re.findall('区(([\u4e00-\u9fa5]+?路)(\w+?号))', addr))  # similar

[('马鞍山南路588号', '马鞍山南路', '588号'), ('莲花路1002号', '莲花路', '1002号')]


### 正则表达式的应用

* 信息查找
* 信息提取
* 信息替换

* 信息查找

  1. 匹配html的一个标签，如title标签、head标签、h1标签。

  2. 匹配一个日期或者时间

  3. 匹配电子邮件地址

  4. 匹配含有区号的电话号码



* 信息提取
  1. 提取title标签中的内容

  2. 提取a标签中的链接地址

  3. 提取日期中的年月日信息

  4. 提取电子邮件的收件人和邮件服务器地址

  5. 提取电话的区号和号码。

## 正则表达式基本语法

### 匹配指定字符串

In [6]:
data3 = 'the dog runs after a cat and a dog'
print(re.findall('dog', data3))
print(re.findall('cat', data3))

['dog', 'dog']
['cat']


### 匹配多种可能的字符串

使用符号'|'，注意选择符'|'的优先级最低，如有必要可以使用圆括号改变优先级。

In [7]:
print(re.findall('dog|cat', data3))

['dog', 'cat', 'dog']


如果要匹配不同形式的yes，该如何处理？

In [8]:
answer = 'Yes'
print(re.findall('yes|Yes|yEs|YEs|yeS|YeS|yES|YES', answer))

['Yes']


### 匹配一类字符：字符类

使用方括号括起来的一系列字符，匹配一个字符的多种可能性，表示一个字符类。

  * [abc] [a-c] 匹配指定字符集或指定范围的字符集
  * [^abc] [^a-c] 排除指定字符集或指定范围的字符集

In [9]:
print(re.findall('[yY][eE][sS]', answer))

['Yes']


In [10]:
# 匹配三位十进制数
print(re.findall('[1-9][0-9][0-9]', '302, 098, 02016, 060599'))
# 匹配两位十六进制数
print(re.findall('[0-9a-fA-F][0-9a-fA-F]', '3469fadc9cdd'))
# 匹配身份证号码后三位
print(re.findall('[0-9][0-9][1-9xX]', '302, 098, 020, 59X'))

['302', '201', '605']
['34', '69', 'fa', 'dc', '9c', 'dd']
['302', '098', '59X']


可以发现以上的模式中，

* 用枚举的形式表示一个字符类，不好理解。

* 若一类字符出现多次，如\[0-9\]，会使得正则表达式变得非常冗长。

In [11]:
# （1）匹配一个非空<x>标签，标签名不知
# not solved
print(re.findall('<[^>]>', '<1> <h4> <title> <>'))
# （2）匹配以t开头但是第二个字母不是元音的词，词的长度为4
# not solved
print(re.findall('t[^aeiouy][a-z][a-z]', 'take this talk stand treat strand'))

['<1>']
['this', 'trea', 't st']


注意以上的模式存在问题：

(1) 虽然排除了空标签，但无法匹配标签名长度可变的问题。

(2) 虽然找到了t后非元音字母的词，但没有限定t出现在词的开头。也不保证找到的是长度为4的词。

### 预定义字符类

| 字符类 | 含义 | 字符类 | 含义 |    
| -- | -- | -- | --|
| \d | 数字[0-9] | \D | 排除数字 |
| \s | 空白字符[ \t\n\r\f\v] | \S | 排除空白字符 |
| \w | 单词字符[a-zA-Z0-9_] | \W | 排除字母字符 |
| . | 除回车符之外的任何字符 |  |  |

In [12]:
# 匹配三位十进制数
print(re.findall('[1-9]\d\d', '302, 098, 02016, 060599'))
# 匹配三位变量名
print(re.findall('[a-zA-Z_]\w\w', 'd_1, psd, _23, _ab, 3_a'))
# 匹配身份证号码后三位
print(re.findall('\d\d[1-9xX]', '302, 098, 020, 59X'))

['302', '201', '605']
['d_1', 'psd', '_23', '_ab']
['302', '098', '59X']


In [13]:
test1 = "the quick brown fox jumps for food"
print(re.findall('\w\w\w', test1))
print(re.findall('\w\w\w\w', test1))

['the', 'qui', 'bro', 'fox', 'jum', 'for', 'foo']
['quic', 'brow', 'jump', 'food']


对于字符类匹配哪些字符存在困惑的同学，可以通过以下的代码对于一定范围内的字符进行测试。

In [14]:
pattern = '\s'
for i in range(128):
    if re.match(pattern, chr(i)):
        print(i,repr(chr(i)), 'match')

9 '\t' match
10 '\n' match
11 '\x0b' match
12 '\x0c' match
13 '\r' match
28 '\x1c' match
29 '\x1d' match
30 '\x1e' match
31 '\x1f' match
32 ' ' match


### 匹配重复的模式

<table>
    <tr><td>后缀</td><td>含义</td> <td>&nbsp;</td> <td>后缀</td><td>含义</td></tr>
    <tr><td>?</td><td>重复0次或1次</td> <td>&nbsp;</td> <td>{n}</td><td>重复n次</td></tr>
    <tr><td>*</td><td>重复0次或多次</td> <td>&nbsp;</td> <td>{n,}</td><td>重复n次或以上</td></tr>
    <tr><td>+</td><td>重复1次或多次</td> <td>&nbsp;</td> <td>{,m}</td><td>重复m次或以下</td></tr>
    <tr> <td>&nbsp;</td><td>&nbsp;</td> <td>&nbsp;</td> <td>{n,m}</td><td>重复至少n次，至多m次</td></tr>

</table>

重复模式举例
<table>
    <tr><td>三位数字</td><td>'\d{3}'</td></tr>
    <tr><td>三位数</td><td>'[1-9]\d{2}'</td></tr>
    <tr><td>时间</td><td>'\d{2}:\d{2}:\d{2}'</td></tr>
    <tr><td>日期</td><td>'\d{4}-\d{2}-\d{2}'</td></tr>
    <tr><td>邮政编码</td><td>'\d{6}'</td></tr>
</table>

In [15]:
# 查找以f开头，后面跟随多个o,最后以一个非o字母结束的模式
# 注意匹配的并不是单词
print(re.findall('fo+[^o]', 'fox   foo find food fun'))

['fox', 'foo ', 'food']


In [16]:
# （1）匹配一个非空<x>标签，标签名不知
# solved
print(re.findall('<[^>]+>', '<1> <h4> <title> <>'))

['<1>', '<h4>', '<title>']


### 匹配边界

| 边界符号 | 匹配边界 | 边界符号 | 匹配边界 |
|--| -- | -- | --|
| ^ | 行的开头 | $ | 行的结尾 | 
| \b |单词边界  | \B | 非单词边界  |
| \A | 字符串开头 | \Z | 字符串结尾 |

In [17]:
print(re.findall('^[A-Z]\w+', "The \nquick \nBrown \nfox \nJumps for \nFood"))  # 默认单行模式
print(re.findall('^[A-Z]\w+', "The \nquick \nBrown \nFox \nJumps for \nFood", re.M))

['The']
['The', 'Brown', 'Fox', 'Jumps', 'Food']


In [18]:
# 应用：检测用户输入的用户名是否合法
if re.match('^\w{3,8}$', 'jackma  20'):  # 把空格去掉试试看
    print('legal')
else:
    print('illegal')

illegal


In [19]:
# （2）匹配以t开头但是第二个字母不是元音的词，词的长度为4
# solved
print(re.findall(r'\bt[^aeiouy]\w{2}\b', 'take this talk stand treat strand'))

['this']


In [20]:
# 查找以f开头的词
test2 = "the quick brown fox jumps for food"
print(re.findall(r'\bf\w+\b', test2))

['fox', 'for', 'food']


### 转义字符

注意到方括号、句点和星号等符号都有特殊的含义，在正则表达式中需要匹配这些字符时，需要进行转义。需要进行转义的符号有：

> . ^ $ * + ? { } [ ] ( ) \ | 


In [21]:
print(re.findall('1+1', '1+1=2'))  # 特殊符号必须转义
print(re.findall('1\+1', '1+1=2'))
print(re.findall('\x66', 'x66=1')) # 不该转义的不能乱转
print(re.findall('x66', 'x66=1'))

[]
['1+1']
[]
['x66']


In [22]:
print(re.findall('3.14', '3.1415926'))  # 目标是找什么？
print(re.findall('3.14', '361415926'))
print(re.findall('3\.14', '361415926'))

['3.14']
['3614']
[]


### 贪婪匹配 vs 懒惰匹配

* 贪婪匹配方式

默认情况下，正则表达式采用贪婪匹配方式。在前导字符后使用星号或加号时，匹配引擎总是尽可能多的重复前导字符。

In [23]:
# 贪婪匹配
re.findall('<.+>', '<td>cell1</td><div>')

['<td>cell1</td><div>']

* 懒惰匹配方式

在重复限定符后面加后缀?，正则表达式匹配引擎使用懒惰匹配方式，总是使用尽可能少的重复前导字符。

In [24]:
# 懒惰匹配方式
re.findall('<.+?>', '<td>cell1</td><div>')

['<td>', '</td>', '<div>']

懒惰匹配方式用于标签识别和标签提取时特别有用。因为一般标记语言的标签不允许使用小于号和大于号进行嵌套。

以下模式也可以用于标签提取，但不等价于懒惰匹配方式。

In [25]:
re.findall('<[^>]+>', '<td>cell1</td><div>')

['<td>', '</td>', '<div>']

In [26]:
re.findall('<.+?>$', '<td>cell1</td><div>')

['<td>cell1</td><div>']

In [27]:
re.findall('<[^>]+>$', '<td>cell1</td><div>')  # 注意两者之间的差别

['<div>']

### 分组与引用

* 分组：使用圆括号指定多个字符，形成一个字符块（组），以便于重复组或提取相关信息。

* 引用：在正则表达式中，引用之前得到的匹配结果。


In [28]:
re.findall('color:(red|blue|green)', 'color:red')

['red']

In [29]:
re.findall('(color:(red|blue|green))', 'color:red')  # 整个正则表达式都在圆括号中

[('color:red', 'red')]

当使用了分组方式后，系统自动将所有匹配的组按顺序编号，存入缓存。第一个匹配的分组编号为1，第二个分组编号为2，以此类推。分组的顺序依据圆括号的起点计算。

若整个正则表达式并不在圆括号中，则其对应于组号为0.

此时，findall返回的是所有的分组(1~n)匹配结果。

In [30]:
re.findall('(\d{4})-(\d{2})-(\d{2})', '2020-11-20')

[('2020', '11', '20')]

In [31]:
re.findall('((\d{4})-(\d{2})-(\d{2}))', '2020-11-20')  # 整个正则表达式都在圆括号中

[('2020-11-20', '2020', '11', '20')]

* 向后引用

比如日期的模式，mm/dd/yyyy, dd/mm/yyyy, yyyy-mm-dd, yyyy.mm.dd

分隔符可以是/或连字符或句点，但中间的分隔符必须一致。这就需要使用引用。

In [32]:
re.findall(r'(\d{4}([\/\-\.])\d{1,2}\2\d{1,2})', '2020-11-20, 2020.11.20, 2020.11/20')

[('2020-11-20', '-'), ('2020.11.20', '.')]

### 匹配选项

* re.A, 使得`\w`等模式只匹配ASCII码字符
* re.I，忽略大小写
* re.M，多行模式，字符串视为多行
* re.S，单行模式，句点可以匹配换行符

In [33]:
print(re.findall('^[A-Z]\w+', "The \nquick \nBrown \nFox \nJumps for \nFood", re.M))

['The', 'Brown', 'Fox', 'Jumps', 'Food']


In [34]:
print(re.findall('[A-Z].{8}', "The \nquick \nBrown \nFox \nJumps for \nFood", re.S))  # 换行符可以用.匹配
print(re.findall('[A-Z].{8}', "The \nquick \nBrown \nFox \nJumps for \nFood", re.M))

['The \nquic', 'Brown \nFo', 'Jumps for']
['Jumps for']


In [35]:
# 应用：匹配用户的应答是否为yes
answer = 'Yes'
if re.match('^yes$', answer, re.I):
    print('yes')
else:
    print('no')

yes


## 正则表达式操作

* 搜索与匹配函数
* 正则表达式编译对象
* Match对象

### 搜索与匹配函数

这些函数有match, search, findall, finditer等，函数签名相似

```
  re.search(pattern, string, flags=0)
```

In [36]:
m = re.match('yes', 'Yes', re.I)
m

<_sre.SRE_Match object; span=(0, 3), match='Yes'>

In [37]:
print(re.match('to', 'we have to be'))
print(re.search('to', 'we have to be'))

None
<_sre.SRE_Match object; span=(8, 10), match='to'>


In [38]:
for i in re.finditer(r'(\d{4}([\/\-\.])\d{1,2}\2\d{1,2})', '2020-11-20, 2020.11.20, 2020.11/20'):
    print(i)

<_sre.SRE_Match object; span=(0, 10), match='2020-11-20'>
<_sre.SRE_Match object; span=(12, 22), match='2020.11.20'>


#### 小结

* re.match或re.search, 若匹配，返回Match对象，否则返回None
  - re.match，从开头开始匹配
  - re.search，从任意位置开始匹配

* re.findall, 返回（分组）匹配结果列表

* re.finditer，返回所有匹配结果的迭代器

### 正则表达式编译对象

使用re.compile可以将正则表达式编译为正则表达式对象，提高匹配的效率。

```
re.compile(pattern, flag=0)
```

In [39]:
tag = re.compile('<[^>]+>')
tag

re.compile(r'<[^>]+>', re.UNICODE)

正则表达式对象的方法search, match, findall, finditer等方法与re模块中的函数基本一致。

In [40]:
tag.findall("<head> <title>first to go</title> </head>")

['<head>', '<title>', '</title>', '</head>']

### Match对象

使用match, search等方法会得到Match对象。Match对象的方法：
* group 返回指定的组
* groups 返回所有组
* start 返回匹配组的开始位置
* end 返回匹配组的结束位置
* span 返回匹配组的位置范围

In [41]:
tag = re.compile('<[^>]+>')
m = tag.search("<head> <title>first to go</title> </head>")
m.groups()

()

In [42]:
tag2 = re.compile('(<([^>]+)>)')
m2 = tag2.search("<head> <title>first to go</title> </head>")
m2.groups()

('<head>', 'head')

### 匹配和替换

re.sub可以用指定内容替换指定模式。
```
  re.sub(pattern, repl, string, count, flags=0)
```

In [43]:
re.sub('dog','cat','the dog chases a cat')

'the cat chases a cat'

### 模式字符串的常见错误

研究一下下面的例子，为何不匹配？

In [44]:
print(re.findall('\bbeginning', 'this is not the beginning'))

[]


In [45]:
print(re.findall('\\n', "the end\\n"))  # why? 猜猜看

[]


`'\b'` 既能在模式表达式里代表单词边界，同时又是一个转义字符backspace.

`'\\n'` 有两个反斜杠，需要对每个反斜杠进行转义。

有两种办法可以解决，将模式中的
  - （1）转义字符前的反斜杠进行转义，
  - （2）使用前缀r''，代表模式为原生字符串，不需要转义。

In [46]:
# solution
print(re.findall('\\bbeginning', 'this is not the beginning'))
print(re.findall(r'\bbeginning', 'this is not the beginning'))

['beginning']
['beginning']


In [47]:
# solution
print(re.findall('\\\\n', "the end\\n"))
print(re.findall(r'\\n', "the end\\n"))

['\\n']
['\\n']


### 细节问题
`'\b'`指单词边界，含义为字母字符和非字母字符的边界

不能简单的将`'\b.+\b]'`中的边界理解为单词的左边界和右边界。

不能保证这样的模式恰好匹配一个单词。看下面的例子:

In [48]:
print(re.findall(r'\bt.{2}\b', 't1,2 t11,2'))

['t1,', 't11']


In [49]:
print(re.findall(r'\bt.+\b', 't1,2 t11,2'))

['t1,2 t11,2']
